In [139]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math, decimal
import requests
dec = decimal.Decimal
import krakenex
from pykrakenapi import KrakenAPI
api = krakenex.API()
k = KrakenAPI(api)
FearGreedUrl = requests.get('https://api.alternative.me/fng/?limit=2').json()

In [122]:
from pathlib import Path
csvpath1 = Path("./full_moon.csv")

In [123]:
full_moon = pd.read_csv(csvpath1, parse_dates=True, index_col=' Date', infer_datetime_format=True)
full_moon.reset_index(inplace=True)
full_moon = full_moon.rename(columns = {' Date':'Full Moon'})
full_moon.drop(['Day',' Time'], axis=1)

,Full Moon
0,1900-01-15
1,1900-02-14
2,1900-03-16
3,1900-04-15
4,1900-05-14
...,...
1863,2050-09-01
1864,2050-09-30
1865,2050-10-30
1866,2050-11-28


In [124]:
ohlc = k.get_ohlc_data('BTCUSD', interval=1440, ascending = True)
Price = ohlc[0]['Price'] = ohlc[0]['close'].rolling(1).mean()
ohlc[0].drop(['high', 'low', 'time', 'open', 'close', 'vwap', 'count', 'volume'], axis = 1).tail()

,Price
dtime,
2022-04-19,41506.2
2022-04-20,41369.9
2022-04-21,40484.1
2022-04-22,39715.9
2022-04-23,39787.9
